In [1]:
import numpy as np
import pandas as pd

import math
import tensorflow as tf
from tensorflow.python.framework import ops
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle

from sklearn.metrics import confusion_matrix, cohen_kappa_score
from sklearn.metrics import f1_score, recall_score

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
!ls

In [2]:
data = pd.read_csv("creditcard.csv")
class_names = {0:'Not Fraud', 1:'Fraud'}
print(data.Class.value_counts().rename(index = class_names))

FileNotFoundError: [Errno 2] File b'creditcard.csv' does not exist: b'creditcard.csv'

In [3]:
print(data.shape)
data


NameError: name 'data' is not defined

In [4]:
X = np.array(data.drop(['Class'],axis = 1))
X

NameError: name 'data' is not defined

In [5]:
def one_hot_matrix(labels , C ):
    C = tf.constant(C , name = "C")
    one_hot_matrix = tf.one_hot(labels , C , axis = 1)
    sess = tf.Session()
    one_hot = sess.run(one_hot_matrix)
    sess.close()
    return one_hot

In [6]:
Y = np.array(data['Class'])
Y

NameError: name 'data' is not defined

In [7]:
from sklearn.model_selection import train_test_split
np.random.seed(123)
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, train_size=0.70, test_size=0.30, random_state=1)


KeyboardInterrupt: 

In [ ]:
X_train = X_Train.T/255
X_test = X_Test.T/255

Y_train = one_hot_matrix(Y_Train,C = 2)  # change into one_hot_matrix
Y_test = one_hot_matrix(Y_Test,C = 2)
print(Y_train[0])

m = X_train.shape[1]
n_x = X_train.shape[0]
n_y = 2
print(m,n_x,n_y,Y_train)

In [ ]:
def ones(shape):
  ones = tf.ones(shape)
  sess = tf.Session()
  ones = sess.run(ones)
  sess.close()
  return ones

In [ ]:
def create_placeholders(n_x, n_y):
  X = tf.placeholder(tf.float32,[n_x,None],name = "X")
  Y = tf.placeholder(tf.float32,[n_y,None],name = "Y")
  return X,Y


In [ ]:
# layer dim
l1 = 500
l2 = 500
l3 = 500
#l4 = 400

In [ ]:
def initialize_parameters(n_x,n_y):
  tf.set_random_seed(1)
  W1 = tf.get_variable("W1",[l1,n_x],initializer = tf.contrib.layers.xavier_initializer(seed = 1))
  b1 = tf.get_variable("b1",[l1,1],initializer = tf.zeros_initializer())
  W2 = tf.get_variable("W2",[l2,l1],initializer = tf.contrib.layers.xavier_initializer(seed = 1))
  b2 = tf.get_variable("b2",[l2,1],initializer = tf.zeros_initializer())
  W3 = tf.get_variable("W3",[l3,l2],initializer = tf.contrib.layers.xavier_initializer(seed = 1))
  b3 = tf.get_variable("b3",[l3,1],initializer = tf.zeros_initializer())
  W4 = tf.get_variable("W4",[n_y,l3],initializer = tf.contrib.layers.xavier_initializer(seed = 1))
  b4 = tf.get_variable("b4",[n_y,1],initializer = tf.zeros_initializer())
  #W5 = tf.get_variable("W5",[n_y,l4],initializer = tf.contrib.layers.xavier_initializer(seed = 1))
  #b5 = tf.get_variable("b5",[n_y,1],initializer = tf.zeros_initializer())
  
  parameters = {"W1":W1,
                "b1": b1,
                "W2":W2,
                "b2": b2,
                "W3":W3,
                "b3": b3,
                "W4": W4,
                "b4": b4,}
  return parameters

In [ ]:
def forward_propagation(X,parameters):
  
  #take parameters from parameters
  W1 = parameters['W1']
  b1 = parameters['b1']
  W2 = parameters['W2']
  b2 = parameters['b2']
  W3 = parameters['W3']
  b3 = parameters['b3']
  W4 = parameters['W4']
  b4 = parameters['b4']
  #W5 = parameters['W5']
  #b5 = parameters['b5']
  
  # neural network--
  z1 = tf.add(tf.matmul(W1,X), b1)
  a1 = tf.nn.relu(z1)
  z2 = tf.add(tf.matmul(W2,a1), b2)
  a2 = tf.nn.relu(z2)
  z3 = tf.add(tf.matmul(W3, a2), b3)
  a3 = tf.nn.relu(z3)
  z4 = tf.add(tf.matmul(W4, a3), b4)
  #a4 = tf.nn.relu(z4)
  #z5 = tf.add(tf.matmul(W5, a4), b5)
  return z4


In [ ]:
def compute_cost(z3, Y):
  logits = tf.transpose(z3)
  labels = tf.transpose(Y)
  ### main commond
  cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits,labels= labels))
  
  return cost
  

In [ ]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):

    m = X.shape[1]                  
    mini_batches = []
    np.random.seed(seed)
    
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:,permutation]
    shuffled_Y = Y[:,permutation]

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:,k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch_Y = shuffled_Y[:,k * mini_batch_size : k * mini_batch_size + mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:,num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:,num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches


In [ ]:
def model(X_train , Y_train , X_test , Y_test , num_epochs , learning_rate , minibatch_size):

	ops.reset_default_graph()

	tf.set_random_seed(1)
	seed = 3
	costs = []
	(n_x , m) = X_train.shape
	n_y = Y_train.shape[0]

	parameters = initialize_parameters(n_x,n_y)
	X , Y = create_placeholders(n_x , n_y)
	Z4 = forward_propagation(X , parameters)
	cost = compute_cost(Z4 , Y)
	optimizer =  tf.train.AdamOptimizer(learning_rate = learning_rate ).minimize(cost)
	init = tf.global_variables_initializer()


	with tf.Session() as sess:
		sess.run(init)
		for epoch in range(num_epochs):
			epoch_cost = 0
			num_minibatches = int(m / minibatch_size)
			seed = seed + 1
			minibatches = random_mini_batches(X_train , Y_train , minibatch_size , seed = 0)

			for minibatch in minibatches:
				(minibatch_X , minibatch_Y) = minibatch
				_ , minibatch_cost = sess.run([optimizer , cost] , feed_dict = {X : minibatch_X , Y : minibatch_Y})
				epoch_cost += minibatch_cost / num_minibatches

			if epoch % 5 == 0:
				costs.append(epoch_cost)
				print("cost after {} epoch:{}".format(epoch , epoch_cost))

		plt.plot(np.squeeze(costs))
		plt.xlabel('cost')
		plt.ylabel('epochs(per 100)')
		plt.title('learning rate='+str(learning_rate))
		plt.show()

		parameters = sess.run(parameters)
		print("parameters tuned")

		correct_prediction = tf.equal(tf.argmax(Z4) , tf.argmax(Y))
		accuracy  = tf.reduce_mean(tf.cast(correct_prediction , float) )

		print("Train accuracy :" , accuracy.eval({X:X_train , Y:Y_train}))
		print("Test accuracy :" , accuracy.eval({X: X_test , Y:Y_test}))


		return parameters


In [ ]:
parameters = model(X_train , Y_train.T, X_test , Y_test.T , num_epochs = 20 , learning_rate = 0.0001 , minibatch_size = 100)


TypeError: ignored